In [34]:
import pyspark
from pyspark import SQLContext
import pyspark.sql.functions as F
import pyspark.sql.types as Types
sc = SparkContext.getOrCreate();
sql = SQLContext(sc)
from pyspark.sql.window import Window

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

simpleData = (("James", "Sales", True), \
    ("Michael", "Sales", True),  \
    ("Michael", "Sales", True),  \
    ("Robert", "Sales", False),   \
    ("Maria", "Finance", None),  \
    ("James", "Sales", False),    \
    ("Scott", "Finance", True),  \
    ("Jen", "Finance", True),    \
    ("Jeff", "Marketing", False), \
    ("Kumar", "Marketing", False),\
    ("Saif", "Sales", True) \
  )
 
columns= ["quote_id", "df_ql_token", "lendable"]
df = spark.createDataFrame(data = simpleData, schema = columns)

cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
df2 = df\
  .groupBy('quote_id', 'df_ql_token')\
    .agg(cnt_cond((F.col('lendable')  == True)).alias("results_lendable"))\
    .withColumn('case_lendable', F.when(F.col('results_lendable') <= 0, False).otherwise(True))\
    .withColumnRenamed("quote_id", "df_case_lendability_quote_id")\
    .withColumnRenamed("df_ql_token", "df_case_lendability_q_token").show()



+----------------------------+---------------------------+----------------+-------------+
|df_case_lendability_quote_id|df_case_lendability_q_token|results_lendable|case_lendable|
+----------------------------+---------------------------+----------------+-------------+
|                         Jen|                    Finance|               1|         true|
|                        Jeff|                  Marketing|               0|        false|
|                     Michael|                      Sales|               2|         true|
|                       James|                      Sales|               1|         true|
|                       Maria|                    Finance|               0|        false|
|                        Saif|                      Sales|               1|         true|
|                       Scott|                    Finance|               1|         true|
|                      Robert|                      Sales|               0|        false|
|         